# Quantum Search for the solution

In [ ]:
import time
import math as m

In [ ]:
import qiskit
import qiskit.tools.jupyter
from qiskit.visualization import *

%qiskit_version_table

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit import Aer
from random import randrange
from qiskit.visualization import plot_state_paulivec, plot_histogram

In [ ]:
from sympy import *
from sympy.logic import simplify_logic

In [ ]:
def symp_aux(nq,ls):
    global i0,i3,i
    nq

    if nq <= 10:
         i, i0, i1, i2, i3, i4, i5, i6, i7, i8 = symbols('i,i0, i1, i2, i3, i4, i5, i6, i7, i8')
    else:
        print('To many qubits')

    lis=[i0, i1, i2, i3, i4, i5, i6, i7, i8]

    def negifzero(x,y):
        if x=='0':
            na = ~y
        else:
            na = y
        return na

    def getand(l):
        for ll in range(len(l)):
            if ll==0:
                na = negifzero(l[ll],i0)
                a = na
            elif ll==1:
                na = negifzero(l[ll],i1)
                a = a & na
            elif ll==2:
                na = negifzero(l[ll],i2)
                a = a & na
            elif ll==3:
                na = negifzero(l[ll],i3)
                a = a & na
            elif ll==4:
                na = negifzero(l[ll],i4)
                a = a & na
            elif ll==5:
                na = negifzero(l[ll],i5)
                a = a & na
            elif ll==7:
                na = negifzero(l[ll],i6)
                a = a & na
            elif ll==8:
                na = negifzero(l[ll],i7)
                a = a & na
            else:
                na = negifzero(l[ll],i8)
                a = a & na

        return a

    def func_aux(i, lis, ls):
        a = i
        for l in range(len(ls)):
            if a != i:
                a = a | getand(ls[l])
            else:
                a = getand(ls[l])
        return a


    fa = func_aux(i, lis, ls)

    sfa = simplify_logic(fa)
    
    return sfa

In [ ]:
def initial(data, ordem):
    N = len(data)
    y = randrange(N-1)
    v_threshold = data[y]
    M = v_threshold + 1
    
    nb=N-1
    qy = nb.bit_length()
    maxq=ordem.bit_length()
    if qy>maxq:
        qaux=qy
    else:
        qaux=maxq
    
    data_b = {}

    for i in range(len(data)): 
        string_index = bin(i)[2:].zfill(qy)
        string_value = bin(data[i])[2:].zfill(maxq)
        data_b[string_index]=string_value
        
    dim = len(data_b[string_index])
    vv=-1
    data_t = {}
    data_simp = {}

    for a in range(dim):
        count=0
        vv=vv+1  
        for x in data_b.values():
            if x[a]=='1':
                vs='v'+str(vv)
                k=data_b
                ps=list(k)[count]
                if vs in data_t:
                    tps.append(ps)
                    data_t[vs]= tps
                else:
                    data_t[vs]= [ps]
                    tps=[ps]
            count=1+count
        if data_t != {}:
            data_simp[vs] = symp_aux(qy, data_t[vs])
    
    return data_simp, qy, maxq, qaux, N, M

### First Cycle

Initialize the memory as $\sum_j \frac{1}{\sqrt{N}} |j\rangle |y\rangle$.

In [ ]:
# create quantum register 
def create_register(qubits_index,qubits_values,qubits_aux):
    qubits_val= qubits_values
    qrn= qubits_index + qubits_val + qubits_aux
    qr = QuantumRegister(qrn, 'qr')
    cr = ClassicalRegister(qubits_index,'cr')
    return qr, cr

In [ ]:
#initialize grover circuit:
def grover_init(qubits, bits, n):
    qc_grover_I = QuantumCircuit(qubits,bits)
    for h in range(n):
        qc_grover_I.h(qubits[h])
    return qc_grover_I

In [ ]:
def explore(expr):
    variables = expr.free_symbols
    list_values = []
    for truth_values in cartes([False, True], repeat=len(variables)):
        values = dict(zip(variables, truth_values))
        values['target']=expr.subs(values)
        list_values.append(values)
    return(list_values)


def truth_table(data,v):
    expr = sympify(data[v]) 
    data_truth=explore(expr)
    return(data_truth)

In [ ]:
def gate_cx(I, qt, qc,qr):
    dataIstr = str(I)
    if dataIstr[0]=='~':
        i= int(dataIstr[2])
        qc.x(qr[i])
        qc.cx(qr[i],qr[qt])
        qc.x(qr[i])
    else:
        i= int(dataIstr[1])
        qc.cx(qr[i],qr[qt])

def gate_toffoli(data, target, qc):
    v = list(data.values())
    k = list(data.keys())
    icf = int(str(k[0])[1])
    ics = int(str(k[1])[1])
    if v[0] and v[1]:
        qc.ccx(qr[icf],qr[ics],qr[target])
    elif v[0] and not(v[1]):
        qc.x(qr[ics])
        qc.ccx(qr[icf],qr[ics],qr[target])
        qc.x(qr[ics])
    elif not(v[0]) and v[1]:
        qc.x(qr[icf])
        qc.ccx(qr[icf],qr[ics],qr[target])
        qc.x(qr[icf])
    else:
        qc.x(qr[icf])
        qc.x(qr[ics])
        qc.ccx(qr[icf],qr[ics],qr[target])
        qc.x(qr[ics])
        qc.x(qr[icf])

In [ ]:
def toffoli_init(c1,c2,target,qc,qr):
    icf = int(str(c1[0])[1])
    ics = int(str(c2[0])[1])
    if c1[1] and c2[1]:
        qc.ccx(qr[icf],qr[ics],qr[target])
    elif c1[1] and not(c2[1]):
        qc.x(qr[ics])
        qc.ccx(qr[icf],qr[ics],qr[target])
        qc.x(qr[ics])
    elif not(c1[1]) and c2[1]:
        qc.x(qr[icf])
        qc.ccx(qr[icf],qr[ics],qr[target])
        qc.x(qr[icf])
    else:
        qc.x(qr[icf])
        qc.x(qr[ics])
        qc.ccx(qr[icf],qr[ics],qr[target])
        qc.x(qr[ics])
        qc.x(qr[icf])
    return target

def toffoli_mid(c1, c2, target, qc,qr):
    ics = int(str(c2[0])[1])
    if c2[1]:
        qc.ccx(qr[c1],qr[ics],qr[target])
    elif not(c2[1]):
        qc.x(qr[ics])
        qc.ccx(qr[c1],qr[ics],qr[target])
        qc.x(qr[ics])
    return target
def toffoli_mid_rev(c1, c2, target, qc,qr):
    ics = int(str(c2[0])[1])
    if c2[1]:
        qc.ccx(qr[c1],qr[ics],qr[target])
    elif not(c2[1]):
        qc.x(qr[ics])
        qc.ccx(qr[c1],qr[ics],qr[target])
        qc.x(qr[ics])
    return target

def decomposed_gate(qcontrol,qtarget,qaux,qc,data,qr):
    di = list(data.items())
    new_control = toffoli_init(di[0], di[1], qaux, qc,qr)
    for x in di[2:-1]:
        qaux=qaux+1
        new_control = toffoli_mid(new_control, x, qaux, qc,qr)
    qc.cx(qr[new_control],qr[qtarget])
    for x in reversed(di[2:-1]):
        qaux=qaux-1
        new_control = toffoli_mid_rev(qaux, x, new_control, qc,qr)-1
    new_control = toffoli_init(di[0], di[1], qaux, qc,qr)

def function_aux(t, qt, qa, qc,qr):
    for x in t:
        control_number = len(x)-1
        if x['target']:
            if control_number==2:
                gate_toffoli(x, qt, qc,qr)
            else:
                decomposed_gate(control_number, qt, qa,qc,x,qr)
                
def grover_indexValues(qubits, bits, n, m, data):
    qc_grover_IV = QuantumCircuit(qubits,bits)
    keys= list(data.keys())
    qt=n
    for k in keys:
        simple=0
        value=data[k]
        # if it is a simbol:
        if type(value)!=And and type(value)!=Or:
            gate_cx(value,qt,qc_grover_IV,qubits)
        #if it is an expression (And / Or)
        else: 
            table = truth_table(data,k)
            function_aux(table, qt, n+m, qc_grover_IV,qubits)
        qt=qt+1
    return(qc_grover_IV)

Mark j if $T[j] < T[y]$.

In [ ]:
def mark_or_zero(v,n,m,qc,qr):
    target = n+m
    control = n 
    for i in v:
        if i=='1':
            qc.cx(qr[control],qr[target])
        control= control+1

def mark_zero(c,t,qc,qr):
    qc.cx(qr[c],qr[t])
    
def mark_other(c,t,qc,qr):
    lc = len(c)
    final_t=t
    if lc == 2:
        qc.x(qr[c[1]])
        qc.ccx(qr[c[0]],qr[c[1]],qr[t])
        qc.x(qr[c[1]])
    else:
        t=t+1
        qc.ccx(qr[c[0]],qr[c[1]],qr[t])
        ic_1=t
        t=t+1
        count=2
        for x in c[2:]:
            count=count+1
            ic_2=x
            if lc==count:
                qc.x(qr[ic_2])
            qc.ccx(qr[ic_1],qr[ic_2],qr[t])
            if lc==count:
                qc.x(qr[ic_2])
            ic_1=t
            t=t+1
        qc.cx(qr[ic_1],qr[final_t])
        #reverse
        t=t-1    
        ic_1=ic_1-1
        for x in reversed(c[1:-1]):
            if lc==count:
                qc.x(qr[ic_2])
            qc.ccx(qr[ic_1],qr[ic_2],qr[t])
            if lc==count:
                qc.x(qr[ic_2])
            ic_1=ic_1-1
            t=t-1
            ic_2=x
            count=count-1
        #t = t-1
        qc.ccx(qr[c[0]],qr[c[1]],qr[t])
        
        
def grover_minor(qubits, bits, n, m, v_threshold):
    qc_grover_m = QuantumCircuit(qubits,bits)    
    v= "{0:b}".format(v_threshold).zfill(m)
    target = m+n
    zero=False
    for x in "{0:b}".format(v_threshold):
        if x == '0':
            zero = True
    if zero==False:
        mark_or_zero(v, n, m, qc_grover_m,qubits)
    else: 
        toconsider = False
        count=n
        lcount= []
        for i in v:
            if i=='1':
                lcount.append(count)
                if not(toconsider):
                    mark_zero(count, target, qc_grover_m,qubits )
                else:
                    mark_other(lcount, target, qc_grover_m,qubits)
                toconsider = True
            count=count+1
    return(qc_grover_m)



In [ ]:
def methods(N, M, method):
    if method=='Grover':
        phy = m.pi
        number_loops = (m.pi / 4)* m.sqrt(N)
        c= int(round(number_loops))
        return c

In [ ]:
def grover_mark_j(qubits, bits, n, m):
    qc = QuantumCircuit(qubits,bits)    
    target = m+n
    final_t = n
    for x in range(n):
        qc.cx(qubits[target], qubits[x])
    if n<= 2:
        qc.cz(qubits[0], qubits[1])
    else:
        t=n+m
        qc.ccx(qubits[0],qubits[1],qubits[t])
        ic_1=t
        for x in range(2,n-1):
            ic_2=x
            t=t+1
            qc.ccx(qubits[ic_1],qubits[ic_2],qubits[t])
            ic_1=t
        qc.cz(qubits[ic_1],qubits[final_t])
        #reverse
        ic_1=ic_1-1
        for x in range(n,3,-1):
            ic_2 =x
            qc.ccx(qubits[ic_1],qubits[ic_2],qubits[t])
            ic_1=t
            t=t-1
        t = t-1
        qc.ccx(qubits[0],qubits[1],qubits[t])
    for x in range(n):
        qc.cx(qubits[target], qubits[x])
    return(qc)


In [ ]:
def decomposed_gate_Z(index,m, qc_grover_D,qubits):
    target= index+m
    qc_grover_D.ccx(qubits[0],qubits[1],qubits[target])
    i=index
    for i in range(2,index-1):
        t=target+1
        qc_grover_D.ccx(qubits[target],qubits[i],qubits[t])
        target=t
    qc_grover_D.cz(qubits[target],qubits[i-1])
    for i in range(index-1,2,-1):
        target=target-1
        t=target+1
        qc_grover_D.ccx(qubits[target],qubits[i-1],qubits[t])
        t=t-1
        target=t
    qc_grover_D.ccx(qubits[0],qubits[1],qubits[target])

def ncz(n,m,qc_grover_D,qubits):
    t=n-1
    if n == 1:
        qc_grover_D.z(qubits[t])
    elif n ==2 :
        qc_grover_D.cz(qubits[0],qubits[t])
    else:
        decomposed_gate_Z(n,m,qc_grover_D,qubits)
    
def grover_D(qubits, bits, n, m):
    qc_grover_D = QuantumCircuit(qubits,bits)
    for i in range(n):
        qc_grover_D.h(qubits[i])
    for i in range(n):
        qc_grover_D.x(qubits[i])
    ncz(n, m, qc_grover_D,qubits)
    for i in range(n):
        qc_grover_D.x(qubits[i])
    for i in range(n):
        qc_grover_D.h(qubits[i])
    return(qc_grover_D)


In [ ]:
#initialize grover circuit:
def grover_measure(qubits, bits, n):
    grover_m= QuantumCircuit(qubits,bits)
    for h in range(n):
        grover_m.measure(qubits[h],bits[h])
    return grover_m


In [ ]:
def grover(qy, maxq, qaux, data_simp,v_threshold,J, monitor=True):
    
    qr, cr = create_register(qy,maxq,qaux)
    qc_grover= QuantumCircuit(qr,cr)
    if monitor == True:
        print('OK registers')
    qc_grover_0 = grover_init(qr, cr, qy)
    if monitor == True:
        print('OK init')
    qc_grover_IV=grover_indexValues(qr,cr,qy,maxq,data_simp)
    if monitor == True:
        print('OK index to values')
    qc_grover_O1=grover_minor(qr, cr, qy, maxq, v_threshold)
    if monitor == True:
        print('OK minor')
    qc_grover_O2=grover_mark_j(qr,cr,qy,maxq)
    if monitor == True:
        print('OK mark j')
    qc_grover_D1=grover_D(qr,cr,qy,maxq)
    if monitor == True:
        print('OK Diffusion')
    qc_grover_M = grover_measure(qr, cr, qy)

    qc_grover = qc_grover_0 + qc_grover_IV
    
    for i in range(J):
        qc_grover = qc_grover + qc_grover_O1 + qc_grover_O2 + qc_grover_D1
    qc_grover = qc_grover + qc_grover_M
    
    return qc_grover

## The Solution

In [ ]:
def findsolution(data,method,backend):
    try:
        top = max(data)
        data_simp, qy, maxq, qaux, N, M = initial(data, top)

        y = randrange(N-1)
        v_threshold = data[y]
        v_threshold_old = top+1

        m0 = int( (45/4)*(m.sqrt(N)) + (7/10)*(m.pow( m.log2(N), 2 )) )

        # method ='Grover'
        J = methods(N,M,method)

        #use qasm_simulator
        backend = Aer.get_backend(backend)
        shots = 1
        exp_counts = []

        print("qy:",qy," maxq:", maxq," qaux:", qaux," N:", N," M:",
              M ," m0:", m0," J:", J,"initial v_threshold: ",v_threshold)

        if qy+maxq+qaux > 32:
            raise Exception('too many qubits')
                
        for i in range(m0):
            print("Iteration number ",i+1," of",m0)
            if (v_threshold != v_threshold_old):
                print("           Making Circuit")
                start = time.time()
                grover_circuit = grover(qy, maxq, qaux, data_simp, v_threshold,J)
                print("           time taken (seconds):",time.time() - start)
                        
            job=execute(grover_circuit, backend, shots=shots)
            print("           Running Circuit")
            start = time.time()
            result_S = job.result()
            print("           time taken (seconds):",time.time() - start)
            
            print("           Getting Results")
            start = time.time()
            counts_sim = result_S.get_counts(grover_circuit)
            print("           time taken (seconds):",time.time() - start)
            exp_counts.append(counts_sim)
            
            y_temp=int(list(counts_sim.keys())[0], 2)
            
            v_threshold_old = v_threshold
            if y_temp < len(data):
                if data[y_temp] <= v_threshold:
                    v_threshold = data[y_temp]
                    a = y_temp
                    print(a,v_threshold)
            
        print('RESULT:', '\n          index',a,'\n          value', v_threshold)
        
        return a, v_threshold, exp_counts
        
    except Exception as e:
        print("Not possible to run, ",e)

# Correspondence between the position and the molecules/conformations
Each position of the list of solutions corresponds to a specific conformation from a specific molecule, with the next function is possible to identify such informations:

In [ ]:
def which2(number,a):
    if(number>=12):
        a += 1
        return which2(number-12,a)
    else:
        print("Conformation from molecule given as input 1: ",a,
              "\nConformation from molecule given as input 2: ",number)
        return [a, number] 

def which3(number,a,b):
    if (number>=144):
        a += 1
        return which3(number-144,a,b)
    else:
        if(number>=12):
            b += 1
            return which3(number-12,a,b)
        else:
            print("Conformation from molecule given as input 1: ",a,
                  "\nConformation from molecule given as input 2: ",b,
                  "\nConformation from molecule given as input 3: ",number)
            return [a, b, number]
        
def whichConfs (number,mol): 
    #insert position of the list and returns to each conformations that corresponds
    if(mol==3):
        numbers = which3(number,0,0)
    if(mol==2):
        numbers = which2(number,0)
    return numbers

## To see solution, insert chosen molecules list of scores as input and it will give as output the positions

#### Import from file

In [ ]:
f=open("input_scores/scores12.txt", "r")
scores12=[]
if f.mode == 'r':
    for number in f:
        scores12.append(int(number))

print("List of Scores:\n",scores12)
print("Minimum of the list: ",min(scores12))

In [ ]:
f=open("input_scores/scores13.txt", "r")
scores13=[]
if f.mode == 'r':
    for number in f:
        scores13.append(int(number))

print("List of Scores:\n",scores13)
print("Minimum of the list: ",min(scores13))

In [ ]:
f=open("input_scores/scores23.txt", "r")
scores23=[]
if f.mode == 'r':
    for number in f:
        scores23.append(int(number))

print("List of Scores:\n",scores23)
print("Minimum of the list: ",min(scores23))

### Data Visualization

In [ ]:
print("Molecules 1, 2")
position12, value12, counts12 = findsolution(scores12,'Grover','qasm_simulator')
whichConfs(position12,2) #this will print the conformations to which the position corresponds

In [ ]:
print("Molecules 1, 3")
position13, value13, counts13 = findsolution(scores13,'Grover','qasm_simulator')
whichConfs(position13,2) #this will print the conformations to which the position corresponds

In [ ]:
print("Molecules 2, 3")
position23,value23, counts23 = findsolution(scores23,'Grover','qasm_simulator')
whichConfs(position23,2) #this will print the conformations to which the position corresponds

#### The next case, of three different molecules to align, does not work due to the higher number of qubits required!!

In [ ]:
f=open("input_scores/scores123.txt", "r")
scores123=[]
if f.mode == 'r':
    for number in f:
        scores123.append(int(number))

print("List of Scores:\n",scores123)
print("Minimum of the list: ",min(scores123))

In [ ]:
print("Molecules 1, 2 and 3")
position13, value13, counts13 = findsolution(scores13,'Grover','qasm_simulator')
whichConfs(position13,2) #this will print the conformations to which the position corresponds